In [1]:
import io
import keras
import tensorflow as tf
import os
import numpy as np # used for handling numbers
import pandas as pd # used for handling the dataset
df= pd.read_csv('drive/My Drive/Colab Notebooks/dataset/labeled_outs.txt',delimiter=",",header=None)
#test_df=pd.read_csv('drive/My Drive/Colab Notebooks/dataset/test_outs.txt',delimiter=",",header=None)
# Dataset is now stored in a Pandas Dataframe

Using TensorFlow backend.


In [0]:
file = open('drive/My Drive/Colab Notebooks/dataset/vocab.txt', 'r')
# read all text
vocab = file.readlines()
vocabulary={}
for w in vocab:
  vocabulary[w.strip()]=""
# close the file
file.close()

In [0]:
df.columns=['label','text']
df.text=df.text.astype(str)
df.label=df.label.astype(str)
df=df.sample(frac=1)
docs=df['text'].values
labels=df['label'].values

In [0]:
docs_filtered=[]
for d in docs:
  # split into tokens by white space
  tokens = d.split()
  temp=""
  for t in tokens:
    if t in vocabulary:
      temp=temp+" "+t
  docs_filtered.append(temp)


In [0]:
# integer encode the documents
from keras.preprocessing.text import one_hot
vocab_size = len(vocabulary)+1

encoded_docs = [one_hot(d, vocab_size) for d in docs_filtered]

In [6]:
# pad documents to a max length of 4 words
max_length = 200
padded_docs = keras.preprocessing.sequence.pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(len(padded_docs))

91795


In [7]:
#import sys
#import numpy
#numpy.set_printoptions(threshold=sys.maxsize)
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(labels)
encoded_Y = encoder.transform(labels)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
print(dummy_y.shape)


(91795, 8)


In [0]:
emb_dim=32
emb_model = tf.keras.Sequential([ 
    tf.keras.layers.Input(shape=(max_length,)) ,                           
    tf.keras.layers.Embedding(vocab_size, output_dim=emb_dim),
    tf.keras.layers.LSTM(128,return_sequences=True,dropout=0.3),
    tf.keras.layers.LSTM(64,return_sequences=True,dropout=0.3),
    tf.keras.layers.LSTM(32,return_sequences=True),
    tf.keras.layers.LSTM(16),
    tf.keras.layers.Dense(8, activation="softmax")
])

emb_model.compile(optimizer='adam',loss= 'categorical_crossentropy',metrics=['acc'])
print(emb_model.summary())


In [0]:
emb_model.fit(padded_docs, dummy_y,epochs=30,batch_size=256, verbose=1,validation_split=0.1)


In [10]:
emb_model1 = tf.keras.Model(inputs=emb_model.inputs , outputs=emb_model.layers[3].output)
emb_X = emb_model1.predict(padded_docs)
print(emb_X.shape)
emb_X=np.reshape(emb_X, (emb_X.shape[0],1,-1))

(91795, 200, 32)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(emb_X, dummy_y, test_size=0.18)

In [0]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


In [12]:
model = tf.keras.Sequential([
#    tf.keras.layers.Embedding(vocab_size, 32),
    tf.keras.layers.LSTM(2048,input_shape=(X_train.shape[1],X_train.shape[2]),return_sequences=True),
    tf.keras.layers.LSTM(1024,return_sequences=True,dropout=0.3,recurrent_dropout=0.3),
    tf.keras.layers.LSTM(512,return_sequences=True,dropout=0.3,recurrent_dropout=0.3),
    tf.keras.layers.LSTM(128,return_sequences=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(8, activation="softmax")
])

In [0]:

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['acc'])
# summarize the model
print(model.summary())

In [14]:
history = model.fit(X_train, y_train, epochs=15,batch_size=256, shuffle=False, verbose=1,validation_split=0.1)
model.save('saved_model/text_classifier_rnn_word_embedding_epoch_20')

Epoch 1/15
265/265 [==============================] - 49s 184ms/step - loss: 0.3097 - acc: 0.9445 - val_loss: 0.1173 - val_acc: 0.9738
Epoch 2/15
265/265 [==============================] - 47s 179ms/step - loss: 0.1192 - acc: 0.9731 - val_loss: 0.1027 - val_acc: 0.9773
Epoch 3/15
265/265 [==============================] - 47s 179ms/step - loss: 0.1085 - acc: 0.9755 - val_loss: 0.0930 - val_acc: 0.9797
Epoch 4/15
265/265 [==============================] - 47s 178ms/step - loss: 0.1028 - acc: 0.9762 - val_loss: 0.1061 - val_acc: 0.9713
Epoch 5/15
265/265 [==============================] - 47s 179ms/step - loss: 0.1009 - acc: 0.9768 - val_loss: 0.1066 - val_acc: 0.9722
Epoch 6/15
265/265 [==============================] - 47s 179ms/step - loss: 0.0997 - acc: 0.9771 - val_loss: 0.0870 - val_acc: 0.9810
Epoch 7/15
265/265 [==============================] - 47s 179ms/step - loss: 0.0950 - acc: 0.9782 - val_loss: 0.0835 - val_acc: 0.9809
Epoch 8/15
265/265 [==============================] - 4

In [0]:
!ls saved_model

text_classifier_rnn_word_embedding_epoch_15
text_classifier_rnn_word_embedding_epoch_20


In [0]:
#model.load_weights('./checkpoints/text_classifier_rnn_word_embedding')
new_model = tf.keras.models.load_model('saved_model/text_classifier_rnn_word_embedding_epoch_20')

In [15]:
results = model.evaluate(X_test, y_test)
print(results)

517/517 [==============================] - 22s 42ms/step - loss: 0.0821 - acc: 0.9808
[0.08205600827932358, 0.9807552695274353]


In [0]:
pred=new_model.predict(padded_docs)
print(type(pred))